In [1]:
import cv2
import numpy as np
from mss import mss

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [2]:
GRAY = True
EDGE = True
SHARPEN = 0
BLUR = 0
PAUSE = False
THRESHOLD_DELTA = 1
BLACKEN_THRESHOLD = 0
NOISE_THRESHOLD = 0
SUPPRESS = True
OLD_SUPPRESS = False
HUMAN = False
MAX_LAST_FRAMES = 10


def convert_gray(frame):
    
    if len(frame.shape) == 2:
        return frame
    
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

def convert_color(frame):
    
    if len(frame.shape) == 3:
        return frame
    
    return cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)

def convolve(frame, kernel=np.array([[0,0,0],[0,1,0], [0,0,0]])):
    
    return cv2.filter2D(frame, -1, kernel)

def edge_detection(frame):
    
    kernel = np.array([
        [-1, -1, -1],
        [-1, 8, -1],
        [-1, -1, -1]
    ])
    
    return convolve(frame, kernel)

def identity():
    
    kernel = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]
    ])
    
    return kernel

def sharpen(frame):
    
    kernel = 1 / 5 * np.array([
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]
    ])
    kernel = (identity() - kernel) * SHARPEN + identity()
    
    return convolve(frame, kernel)

def blur(frame):
    
    kernel = 1/16 * np.array([
        [1, 2, 1],
        [2, 4, 2],
        [1, 2, 1]
    ])
    
    return convolve(frame, kernel)

def suppress(frame, threshold):
    
    below_threshold = (frame < threshold)
    above_threshold = (frame >= threshold)
    frame = frame - frame * below_threshold + (255 - frame ) * above_threshold 
    
    return frame.astype(np.uint8)

def old_suppress(frame, threshold):

    frame = frame * (1 - (frame < threshold))
    
    return frame.astype(np.uint8)

def remove_noise(frame, threshold):
    
    kernel = np.ones([5, 5]) / 25
    
    mask = 1 - (convolve(frame, kernel) < threshold)
    
    frame = frame * mask
    return frame.astype(np.uint8)
    
    
def key_loop():
    global GRAY
    global EDGE
    global SHARPEN
    global BLUR
    global PAUSE
    global BLACKEN_THRESHOLD
    global SUPPRESS
    global THRESHOLD_DELTA
    global OLD_SUPPRESS
    global NOISE_THRESHOLD
    global HUMAN
    global MAX_LAST_FRAMES

    
    if key == ord('g'):
        GRAY = not GRAY
        print(f'Gray: {GRAY}')

    if key == ord('e'):
        EDGE = not EDGE
        print(f'Edge: {EDGE}')
        
    if key == ord('t'):
        BLACKEN_THRESHOLD = min(BLACKEN_THRESHOLD + THRESHOLD_DELTA, 255)
        print(f'Suppress Threshold: {BLACKEN_THRESHOLD}')
        
    if key == ord('r'):
        BLACKEN_THRESHOLD = max(BLACKEN_THRESHOLD - THRESHOLD_DELTA, 0)
        print(f'Suppress Threshold: {BLACKEN_THRESHOLD}')

    if key == ord('s'):
        SHARPEN += 1
        print(f'Sharpen: {SHARPEN}')

    if key == ord('a'):
        SHARPEN = max(0, SHARPEN - 1)
        print(f'Sharpen: {SHARPEN}')

    if key == ord('b'):
        BLUR += 1
        print(f'Blur: {BLUR}')

    if key == ord('v'):
        BLUR = max(0, BLUR - 1)
        print(f'Blur: {BLUR}')
        
    if key == ord('c'):
        SUPPRESS = not SUPPRESS
        print(f'Suppress: {SUPPRESS}')
    
    if key == ord('o'):
        OLD_SUPPRESS = not OLD_SUPPRESS
        print(f'Old Suppress: {OLD_SUPPRESS}')
    
    if key == ord('u'):
        MAX_LAST_FRAMES = MAX_LAST_FRAMES + 1
    if key == ord('y'):
        MAX_LAST_FRAMES = max(MAX_LAST_FRAMES - 1, 0)
        
    if key == ord('h'):
        HUMAN = not HUMAN
        print(f'Human Detection: {HUMAN}')

def detect_human(frame):
    global GRAY
    
    if not GRAY:
        frame = convert_gray(frame)
    
    boxes, weights = hog.detectMultiScale(frame, winStride=(8,8) )

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])
    
    for (xA, yA, xB, yB) in boxes:
        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB),
                          (0, 255, 0), 2)
    
    return frame.astype('uint8')

def brightness(frame):
    
    if len(frame.shape) > 2:
        frame = convert_gray(frame)
    
    return frame.mean()

def combine_frames(frame, last_frames):
    
    frame = convert_color(frame)
    for index, alt_frame in enumerate(last_frames[::-1]):
        gray_frame = convert_gray(frame)
        gray_alt_frame = convert_gray(alt_frame)
            
        frame_mask = 0.5 * (gray_frame > 0)
        alt_mask = 1.0 * (gray_alt_frame > 0)
        
        alt_mask = abs(alt_mask - frame_mask) > 0.5
        alt_mask = np.dstack([alt_mask, alt_mask , alt_mask]).reshape(alt_mask.shape[0],alt_mask.shape[1],3)
        
        frame = frame + convert_color(alt_frame) * (alt_mask * (0.99 ** index)).astype(np.uint8)
        
    return frame.astype('uint8')
        

def loop(frame, out, key, last_frames):
    global GRAY
    global EDGE
    global SHARPEN
    global BLUR
    global PAUSE
    global BLACKEN_THRESHOLD
    global SUPPRESS
    global OLD_SUPPRESS
    global NOISE_THRESHOLD
    global HUMAN
    
    if key == ord('q'):
        return False
    
    if PAUSE:
        display_frame = combine_frames(frame, last_frames)
        out.write(display_frame)
        cv2.imshow('Camera', display_frame)
        return True, frame

    key_loop()
    
    if GRAY:
        frame = convert_gray(frame)
        
    if HUMAN:
        frame = detect_human(frame)

    if EDGE:
        frame = edge_detection(frame)

    if SHARPEN > 0:
        frame = sharpen(frame)

    if BLUR > 0:
        for i in range(BLUR):
            frame = blur(frame)
        
    if SUPPRESS and not OLD_SUPPRESS:
        frame = suppress(frame, BLACKEN_THRESHOLD)
        frame = remove_noise(frame, NOISE_THRESHOLD)
    
    if SUPPRESS and OLD_SUPPRESS:
        frame = old_suppress(frame, BLACKEN_THRESHOLD)
        frame = remove_noise(frame, NOISE_THRESHOLD)

    if key == ord('x'):
        print(convert_gray(frame).mean())
        
    display_frame = combine_frames(frame, last_frames)

    if out is not None:
        out.write(display_frame)

    cv2.imshow('Camera', display_frame)
    
    if SUPPRESS:
        average_brightness = brightness(frame)
        if average_brightness <= 1.25 and NOISE_THRESHOLD > 0:
            NOISE_THRESHOLD = max(NOISE_THRESHOLD - THRESHOLD_DELTA, 0)
            print(f'Noise Threshold: {NOISE_THRESHOLD}')
        if average_brightness >= 1.75 and NOISE_THRESHOLD < 255:
            NOISE_THRESHOLD = min(NOISE_THRESHOLD + THRESHOLD_DELTA, 255)
            print(f'Noise Threshold: {NOISE_THRESHOLD}')
        if average_brightness <= 1.25 and BLACKEN_THRESHOLD > 0:            
            BLACKEN_THRESHOLD = max(BLACKEN_THRESHOLD - THRESHOLD_DELTA, 0)
            print(f'Suppress Threshold: {BLACKEN_THRESHOLD}')
        if average_brightness >= 1.75 and BLACKEN_THRESHOLD < 255:
            BLACKEN_THRESHOLD = min(BLACKEN_THRESHOLD + THRESHOLD_DELTA, 255)
            print(f'Suppress Threshold: {BLACKEN_THRESHOLD}')
    
    return True, frame

In [ ]:


cam = cv2.VideoCapture(0)
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('output.avi', fourcc, 15.0, (frame_width, frame_height))

last_frames = []
keep_going = True
current_exception = None
while keep_going:
    key = cv2.waitKey(1)
    if key == ord('p'):
        PAUSE = not PAUSE
        print(f'Pause: {PAUSE}')
    try:
        ret, frame = cam.read()
        if ret: 
            if PAUSE and last_frames:
                frame = last_frames[-1]
            keep_going, last_frame = loop(frame, out, key, last_frames)
            last_frames.append(last_frame)
            while len(last_frames) > MAX_LAST_FRAMES:
                last_frames.pop(0)
            
        current_exception = None
    except Exception as exc:
        
        if key == ord('q'):
            keep_going=False
        
        if current_exception is None:
            print(exc)
            current_exception = exc
        elif str(current_exception) == str(exc):
            continue
        else:
            print(exc)
            current_exception = exc
else:
    cam.release()
    out.release()
    cv2.destroyAllWindows()


Noise Threshold: 1
Suppress Threshold: 1
Noise Threshold: 2
Suppress Threshold: 2
Noise Threshold: 3
Suppress Threshold: 3
Noise Threshold: 4
Suppress Threshold: 4
Noise Threshold: 5
Suppress Threshold: 5
Noise Threshold: 6
Suppress Threshold: 6
Noise Threshold: 7
Suppress Threshold: 7
Noise Threshold: 8
Suppress Threshold: 8
Noise Threshold: 9
Suppress Threshold: 9
Noise Threshold: 10
Suppress Threshold: 10
Noise Threshold: 11
Suppress Threshold: 11
Noise Threshold: 12
Suppress Threshold: 12
Noise Threshold: 13
Suppress Threshold: 13
Noise Threshold: 14
Suppress Threshold: 14
Noise Threshold: 15
Suppress Threshold: 15
Noise Threshold: 16
Suppress Threshold: 16
Noise Threshold: 17
Suppress Threshold: 17
Noise Threshold: 18
Suppress Threshold: 18
Noise Threshold: 19
Suppress Threshold: 19
Noise Threshold: 20
Suppress Threshold: 20
Noise Threshold: 21
Suppress Threshold: 21
Noise Threshold: 22
Suppress Threshold: 22
Noise Threshold: 23
Suppress Threshold: 23
Noise Threshold: 24
Suppress 

Noise Threshold: 83
Suppress Threshold: 83
Noise Threshold: 84
Suppress Threshold: 84
Noise Threshold: 85
Suppress Threshold: 85
Noise Threshold: 84
Suppress Threshold: 84
Noise Threshold: 83
Suppress Threshold: 83
Noise Threshold: 84
Suppress Threshold: 84
Noise Threshold: 85
Suppress Threshold: 85
Noise Threshold: 86
Suppress Threshold: 86
Noise Threshold: 87
Suppress Threshold: 87
Noise Threshold: 88
Suppress Threshold: 88
Noise Threshold: 87
Suppress Threshold: 87
Noise Threshold: 86
Suppress Threshold: 86
Noise Threshold: 87
Suppress Threshold: 87
Noise Threshold: 86
Suppress Threshold: 86
Noise Threshold: 85
Suppress Threshold: 85
Noise Threshold: 84
Suppress Threshold: 84
Noise Threshold: 83
Suppress Threshold: 83
Noise Threshold: 82
Suppress Threshold: 82
Noise Threshold: 83
Suppress Threshold: 83
Noise Threshold: 82
Suppress Threshold: 82
Noise Threshold: 81
Suppress Threshold: 81
Noise Threshold: 80
Suppress Threshold: 80
Noise Threshold: 79
Suppress Threshold: 79
Noise Thres

Noise Threshold: 82
Suppress Threshold: 82
Noise Threshold: 83
Suppress Threshold: 83
Noise Threshold: 82
Suppress Threshold: 82
Noise Threshold: 83
Suppress Threshold: 83
Noise Threshold: 82
Suppress Threshold: 82
Noise Threshold: 81
Suppress Threshold: 81
Noise Threshold: 80
Suppress Threshold: 80
Noise Threshold: 79
Suppress Threshold: 79
Noise Threshold: 78
Suppress Threshold: 78
Noise Threshold: 77
Suppress Threshold: 77
Noise Threshold: 76
Suppress Threshold: 76
Noise Threshold: 75
Suppress Threshold: 75
Noise Threshold: 74
Suppress Threshold: 74
Noise Threshold: 73
Suppress Threshold: 73
Noise Threshold: 72
Suppress Threshold: 72
Noise Threshold: 71
Suppress Threshold: 71
Noise Threshold: 70
Suppress Threshold: 70
Noise Threshold: 69
Suppress Threshold: 69
Noise Threshold: 68
Suppress Threshold: 68
Noise Threshold: 67
Suppress Threshold: 67
Noise Threshold: 66
Suppress Threshold: 66
Noise Threshold: 65
Suppress Threshold: 65
Noise Threshold: 64
Suppress Threshold: 64
Noise Thres

Noise Threshold: 101
Suppress Threshold: 101
Noise Threshold: 102
Suppress Threshold: 102
Noise Threshold: 103
Suppress Threshold: 103
Noise Threshold: 104
Suppress Threshold: 104
Noise Threshold: 105
Suppress Threshold: 105
Noise Threshold: 104
Suppress Threshold: 104
Noise Threshold: 103
Suppress Threshold: 103
Noise Threshold: 102
Suppress Threshold: 102
Noise Threshold: 103
Suppress Threshold: 103
Noise Threshold: 104
Suppress Threshold: 104
Noise Threshold: 105
Suppress Threshold: 105
Noise Threshold: 106
Suppress Threshold: 106
Noise Threshold: 107
Suppress Threshold: 107
Noise Threshold: 108
Suppress Threshold: 108
Noise Threshold: 109
Suppress Threshold: 109
Sharpen: 6
Noise Threshold: 108
Suppress Threshold: 108
Sharpen: 5
Noise Threshold: 107
Suppress Threshold: 107
Noise Threshold: 106
Suppress Threshold: 106
Sharpen: 4
Noise Threshold: 105
Suppress Threshold: 105
Noise Threshold: 104
Suppress Threshold: 104
Sharpen: 3
Noise Threshold: 103
Suppress Threshold: 103
Noise Thres

Noise Threshold: 60
Suppress Threshold: 60
Noise Threshold: 61
Suppress Threshold: 61
Noise Threshold: 62
Suppress Threshold: 62
Noise Threshold: 63
Suppress Threshold: 63
Noise Threshold: 64
Suppress Threshold: 64
Noise Threshold: 65
Suppress Threshold: 65
Noise Threshold: 66
Suppress Threshold: 66
Noise Threshold: 67
Suppress Threshold: 67
Noise Threshold: 68
Suppress Threshold: 68
Noise Threshold: 69
Suppress Threshold: 69
Noise Threshold: 70
Suppress Threshold: 70
Noise Threshold: 71
Suppress Threshold: 71
Noise Threshold: 72
Suppress Threshold: 72


In [ ]:
BLACKEN_THRESHOLD = 10
below_threshold = (last_frame < BLACKEN_THRESHOLD)
above_threshold = (last_frame >= BLACKEN_THRESHOLD)
suppressed = last_frame - (last_frame * below_threshold) + (255 - last_frame * above_threshold)
suppressed

In [ ]:
last_frame - last_frame * below_threshold + (255 - last_frame ) * above_threshold 

with mss() as sct:
    out = None

    last_frame = None
    keep_going = True
    current_exception = None
    while keep_going:
        frame = np.array(sct.grab(sct.monitors[2]))
        
        key = cv2.waitKey(1)
        if key == ord('p'):
            PAUSE = not PAUSE
            print(f'Pause: {PAUSE}')
        try:
            ret = True
            if ret: 
                if PAUSE:
                    frame = last_frame
                keep_going, last_frame = loop(frame, out, key)

            current_exception = None
        except Exception as exc:

            if key == ord('q'):
                keep_going=False
                break

            if current_exception is None:
                print(exc)
                current_exception = exc
            elif str(current_exception) == str(exc):
                continue
            else:
                print(exc)
                current_exception = exc
    else:
        cam.release()
        out.release()
        cv2.destroyAllWindows()
